## Import Important LIB

In [ ]:
import os
import json
from langchain.chat_models import AzureChatOpenAI #ต้องเป็นAzureChatOpenAI

In [49]:
from azure.ai.inference.models import (
    ChatCompletionsToolDefinition,
    FunctionDefinition,
    UserMessage,
)

## Define Function call > Tools

In [8]:
from langchain.tools import tool

In [80]:
import requests
@tool
def get_person_detail(person_id):
    """สำหรับการค้นชื่อ,เบอร์โทรติดต่อของลูกค้าจากฐานข้อมูลที่มีจาก id ของuser"""
    BASE_URL = "http://127.0.0.1:8000" #ใช้จริงอาจจะต้องเปลี่ยน URL
    response = requests.get(f"{BASE_URL}/get_customer/{person_id}")
    if response.status_code == 200:
        that_Cus = response.json()
        if that_Cus:
            Cus_Name=that_Cus['Name']
            Cus_tel=that_Cus['Telephone']
        else:
            print("Not found that person")
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return f'ลูกค้าชื่อ {Cus_Name} เบอร์โทรศัพท์คือ {Cus_tel} '

In [83]:
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta

# @tool #มันไม่ให้ใช้@Tool ตอนเทส เลยขอcommentไว้ก่อน
def Aproximate_usage(Latest_buy, Cus_Instock):
    """ช่วยในการคำนวนการบริโภคของลูกค้า"""
    #BASE_URL ตัวนี้จะไปเอาวันเดือนปี ปัจจุบันมาคำนวน
    BASE_URL = "https://tools.aimylogic.com/api/now?tz=Asia/Istanbul&format=dd/MM/yyyy"
    response = requests.get(BASE_URL)
    if response.status_code != 200:
        print("Error fetching current date:", response.text)
        return "ไม่สามารถดึงวันที่ปัจจุบันได้"

    output = response.json()
    # print("API Response:", output)  # Debugging

    date_str = output.get('formatted', None) 

    try:
        date_obj = datetime.strptime(date_str, "%d/%m/%Y").date()
        Latest_buy_obj = datetime.strptime(Latest_buy, "%Y-%m-%d").date()
        diff = relativedelta(date_obj, Latest_buy_obj) 
        months_diff = diff.years * 12 + diff.months

        # print(f"Latest Buy: {Latest_buy_obj}, Current Date: {date_obj}, Months Diff: {months_diff}")  
        if months_diff < 2:
            return 'ลูกค้ามีสินค้าคงคลังอยู่'
        if months_diff >= 2 or Cus_Instock <= 4:
            return 'ลูกค้ามีจำนวนคงคลังน้อยกว่า 1 เดือนแล้ว แนะนำโปรโมชั่น'
        else:
            return 'ลูกค้ามีสินค้าคงคลังอยู่'
    
    except Exception as e:
        print("Error in date processing:", str(e))
        return "เกิดข้อผิดพลาดในการคำนวณ"

In [85]:
# @tool #มันไม่ให้ใช้@Tool ตอนเทส เลยขอcommentไว้ก่อน
def get_promotion():
    """ให้โปรโมชั่นที่เหมาะสมกับลูกค้า"""
    #คิดว่าถ้ามีเป็นตารางเก็บโปรโมชั่นแต่ละเดือนน่าจะ practicalกว่า แถมน่าจะ Adjustอะไรได้มากกว่าด้วย 
    #อาจจะรับจำนวนกระปุกที่เหลือ และใช้ url 2 ตัว ,1 คือurlรับวันเดือน ,อีก 1 คือurlค้นหาโปรโมชั่นประจำเดือน ของกลุ่มคนที่มีความเสี่ยงที่แตกต่าง
    return 'ซื้อ 3 แถม 2! ซื้อ 5 ลดทันที 50%! แคปซูลลดโคเลสเตอรอล ลดไขมันในเลือด ปรับสมดุลสุขภาพ ลดความเสี่ยงโรคหัวใจ รีบสั่งซื้อเลย!'

In [ ]:
# @tool #มันไม่ให้ใช้@Tool ตอนเทส เลยขอcommentไว้ก่อน
def store_user_detail(customer_data):
    """เก็บข้อมูลของลูกค้าลง database"""
    BASE_URL = "http://127.0.0.1:8000/post_customer"
    response = requests.post(BASE_URL, json=customer_data)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to store user details", "status_code": response.status_code}


In [90]:
# !fastapi dev api.py
# get_person_detail('1')
Aproximate_usage("2024-12-25", 6)
get_promotion()

'ซื้อ 3 แถม 2! ซื้อ 5 ลดทันที 50%! แคปซูลลดโคเลสเตอรอล ลดไขมันในเลือด ปรับสมดุลสุขภาพ ลดความเสี่ยงโรคหัวใจ รีบสั่งซื้อเลย!'